<a href="https://colab.research.google.com/github/Chrostowska/Code/blob/Notes-TF/Lesson_2_Training_Process_Linear_regression_%26_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sklearn


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
%tensorflow_version 2.x #for work with online notebook

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output #for that notebook
from six.moves import urllib #for this notebook

import tensorflow.compat.v2.feature_column as fc #feature columns
import tensorflow as tf

In [ ]:
#we need to split dataset before on 2 datasets so alghoritm will not seen datas for validation


dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
#print(dftrain.head()) we have all columns
y_train = dftrain.pop('survived') #deleting column survived and safe it in variable y_train
y_eval = dfeval.pop('survived')


In [ ]:
#PREPROCESSING OF DATAS

CATEGORICAL_COLUMNS = ['sex', 'class', 'deck', 'embark_town', 'alone', 'n_siblings_spouses', 'parch']
NUMERIC_COLUMNS = ['age', 'fare'] # 'n_siblings_spouses', 'parch' are numeric but not with a datatype int, not float

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique() # gets a list of all unique values from given feature columns
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)) #Preprocessing of datas from categorical to numerical

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype = tf.float32))

print(feature_columns)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, def

In [ ]:
dftrain['embark_town'].unique()

array(['Southampton', 'Cherbourg', 'Queenstown', 'unknown'], dtype=object)

In [ ]:
# TRAINING PROCESS
'''batches = number of portions of datas
epochs = how many times model going to see the same datas (loading datas in different order as for a first time model for ex. linear regression could not look like the best fit)
starting with lower epochs, otherwise we'll get overfitted model

Input funtion - defines how dataset will be converted into batches at each epoch
input function convert pandas data frame into dataset object as TensorFlow model is requiring tf.data.Dataset object'''

#data_df - pandas dataframe
#label_df - y_train, y_eval
#ds - data set
#tf.data.Dataset - tensorframe data Dataset
#input_fn - input function


# INPUT FUNCTION

def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function(): # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df)) #create tf.data.Dataset object with data and it's label
    if shuffle:
      ds = ds.shuffle(1000) #randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs) #split dataset into batches of 32 and repeat process for number of epochs
    return ds #return a batch of the dataset
  return input_function #return a function object for use


train_input_fn = make_input_fn(dftrain, y_train) #here we'll call input function that was returned to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)


In [ ]:
#inspect the dataset:

ds = make_input_fn(dftrain, y_train, batch_size=10)()
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of class:', feature_batch['class'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

  age_column = feature_columns[7]
tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy()

gender_column = feature_columns[0]
tf.keras.layers.DenseFeatures([tf.feature_column.indicator_column(gender_column)])(feature_batch).numpy()

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


Some feature keys: ['sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

A batch of class: [b'Third' b'Third' b'Second' b'First' b'Second' b'Second' b'Third'
 b'First' b'Third' b'First']

A batch of Labels: [0 0 1 1 1 1 0 0 0 1]


array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
#CREATING THE MODEL

#linear regression using for predicting numeric values

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns) #creating linear estimator by passing feature columns created earlier
#linear_est is the name of the model
linear_est.train(train_input_fn) #train
result = linear_est.evaluate(eval_input_fn) #get model metric/stats by testing on testing data

clear_output() #clear consoke output
print(result['accuracy']) #the result variable is simply a dict of stats about our model

0.74242425


In [ ]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[4])
print(y_eval.loc[4])
print(result[4]['probabilities'][1]) #last 0 is for not survive, changed on 1 is survive

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


sex                          male
age                          34.0
n_siblings_spouses              0
parch                           0
fare                         13.0
class                      Second
deck                            D
embark_town           Southampton
alone                           y
Name: 4, dtype: object
1
0.28283432


In [ ]:
'''CLASSIFICATION'''

# used to separate data points into classes of different labels

%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
from IPython.display import clear_output #for that notebook
from six.moves import urllib #for this notebook
import tensorflow as tf


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species'] #inf about flower
SPECIES = ['Setosa', 'Versicolor', 'Virginica'] #3 classes of species

train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header = 0)

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')

train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
train.shape

(120, 4)

In [ ]:
def input_fn(features, labels, training = True, batch_size = 256): #Convert the inout into dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #shuffle and repeat if in training moode
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [ ]:
#Feature columns describe how to use the input

my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key = key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
#BUILDING THE MODEL (estimator)
'''
for classification tasks a lot of different estimators/models eg:

  DNNClassifier (Deep Neural Network)
  LinearClassifier

  tf.estimator.DNNClassifier for deep models that perform multi-class classification.
  tf.estimator.DNNLinearCombinedClassifier for wide & deep models.
  tf.estimator.LinearClassifier for classifiers based on linear models.

  '''

#Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each
classifier = tf.estimator.DNNClassifier(feature_columns = my_feature_columns,
#Two hidden layers of 30 and 10 nodes respectively.
hidden_units = [30, 10],
# The model must choose between 3 classes of species which is: ['Setosa', 'Versicolor', 'Virginica']
n_classes = 3)



#Train, Evaluate, and Predict

#Train
classifier.train(
    input_fn = lambda : input_fn(train, train_y, training = True),
    steps = 5000) #less steps in result, better

In [ ]:
#Evaluate the trained model
eval_result = classifier.evaluate(input_fn = lambda : input_fn(test, test_y, training = False))
#lambda - anonymous function that can be defined in one line: x = lambda : print('hi') and then x() it's calling that function
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.533



In [ ]:
#Making predictions (inferring) from the trained model

# Generate predictions from the model
Expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength' : [5.1, 5.9, 6.9],
    'SepalWidth' : [3.3, 3.0, 3.1],
    'PetalLength' : [1.7, 4.2, 5.4],
    'PetalWidth' : [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size = 256):
  """An input function for prediction."""
  # Convert the inputs to a Dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

  predictions = classifier.predict(
      input_fn = lambda : input_fn(predict_x))


In [ ]:
def input_fn(features, batch_size = 256):
  """An input function for prediction."""
  # Convert the inputs to a Dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print('Please type numeric value as prompted')
for feature in features:
  valid = True
  while valid:
    val = input(feature + ':')
    if not val.isdigit() : valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda : input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))



Please type numeric value as prompted
SepalLength:3.0
SepalWidth:1.0
PetalLength:3.0
PetalWidth:0.5
Prediction is "Virginica" (48.8%)
